In [ ]:
# !pip install SpeechRecognition
# !pip install soundfile
# !pip install sounddevice

## 참고자료
- 쓰레딩 https://docs.python.org/ko/3/library/threading.html

In [ ]:
## 디바이스 채널 확인
import sounddevice as sd
sd.query_devices()

#### simple recording + recognition 테스팅

In [ ]:


import queue, os, threading
import sounddevice as sd
import soundfile as sf
import time
from scipy.io.wavfile import write
import speech_recognition as sr

q = queue.Queue()
recorder = False
recording = False

def complicated_record():
    with sf.SoundFile("./micInputTest.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=complicated_save):
            while recording:
                file.write(q.get())
        
def complicated_save(indata, frames, time, status):
    q.put(indata.copy())
    
def start():
    global recorder
    global recording
    recording = True
    recorder = threading.Thread(target=complicated_record)
    print('start recording')
    recorder.start()
    
def stop():
    global recorder
    global recording
    recording = False
    recorder.join()
    print('stop recording')
    
start()
time.sleep(5)
stop()

# Initialize the Recognizer
r = sr.Recognizer()
sentence = ""
try :
    audioFile = sr.AudioFile('./micInputTest.wav')
    with audioFile as source :
        audio = r.record(source)
    sentence = r.recognize_google(audio, language='ko-KR')
    
except :
    print("no input")
    
print(sentence)

In [ ]:
## import, variables, 

import queue, threading
import sounddevice as sd
import soundfile as sf
import time
import numpy as np
from scipy.io.wavfile import read, write
import speech_recognition as sr

import matplotlib.pyplot as plt
import wave, sys 

# Initialize the Recognizer
r = sr.Recognizer()
q = queue.Queue()

silence_threshold = 0.5 
silence_dur = 5
silence_start = 0
silence_end = 0

## global 
recorder = False
recording = False
recorder2 = False
recording2 = False
answered = False
trigger_word = "짱구"  


### 이름 체크 (5초 간격으로)

In [ ]:


def trigCheckFile():
    with sf.SoundFile("./trigCheck.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=micSave):
            while recording:
                file.write(q.get())
        
def micSave(indata, frames, time, status):
    q.put(indata.copy())  

def checkingTrigger(name) :
    while True :
        global recording
        global recorder
        
        recording = True
        recorder = threading.Thread(target=trigCheckFile )
        print('trigcheck : start recording')
        recorder.start()

        time.sleep(3)

        recording = False
        recorder.join()
        print('trigcheck : stop recording')

        audioFile = sr.AudioFile('./trigCheck.wav')
        try :
            with audioFile as source :
                audio = r.record(source)

            print("recognizing...")
            print(r.recognize_google(audio, language='ko-KR'))
            recognized = r.recognize_google(audio, language='ko-KR')

            if name in recognized :
                print("trigcheck : 이름 트리거")
                return 
        except :
            print("trigcheck : no input")
    return


### get user sentence + Silence check
- 문장의 첫 음절을 말하기 전까지는 타이머 시작 안함.
- threshold 를 넘어가는 소리가 입력됨 = 타이머 시작시간 리셋
- 시작시간부터 현재까지(현재 시간 계속 업데이트) 가 dur를 넘어가면 recording 종료

In [ ]:
recorder_test = False
recording_test = False
start = 0
end = 0
dur = 2
thr = 10

listams = []




def micSave(indata, frames, time, status):
    q.put(indata.copy())  

def inputUserSentence():
    global recording_test
    global start
    global end
    isTalking = False
    
    
    with sf.SoundFile("./test.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=micSave):
            print("gettinginput")

            while recording_test :
                audio_data = q.get()
                file.write(audio_data)

                ## 소리 볼륨 한계치 20 아래면 silence
                ams = np.sqrt(np.mean(abs(audio_data)))
                listams.append(ams)
                # 현재 시간
                end = time.time()
                
                ## 문장 시작
                if not isTalking and ams > thr :
                    print("sentence : sentence start")
                    isTalking = True
                    start = time.time()
                    
                ## 이야기중임. 
                if ams > thr  and isTalking :
                    start = time.time()
                    print("sentence : startTime updated")
                
                ## 문장 끝나고 dur 만큼 조용함.
                if end - start >= dur and isTalking:
                    print(f"sentence : silence for {dur} sec. end recording")
                    recording_test = False
                    
                
def getSentence():
    global recorder_test
    global recording_test
    
    recording_test = True
    recorder_test = threading.Thread(target=inputUserSentence)
    print('sentence : getSentence start recording')
    recorder_test.start()
    
    
    time.sleep(10)
    print("sentence : getSentence done")
    recording_test = False
    recorder_test.join()   
                

getSentence()

print("end")


In [ ]:


  
# shows the sound waves 
def visualize(path: str): 
    
    # reading the audio file 
    raw = wave.open(path) 
      
    # reads all the frames  
    # -1 indicates all or max frames 
    signal = raw.readframes(-1) 
    signal = np.frombuffer(signal, dtype ="int16") 
      
    # gets the frame rate 
    f_rate = raw.getframerate() 
  
    # to Plot the x-axis in seconds  
    # you need get the frame rate  
    # and divide by size of your signal 
    # to create a Time Vector  
    # spaced linearly with the size  
    # of the audio file 
    time = np.linspace( 
        0, # start 
        len(signal) / f_rate, 
        num = len(signal) 
    ) 
  
    # using matplotlib to plot 
    # creates a new figure 
    plt.figure(1) 
      
    # title of the plot 
    plt.title("Sound Wave") 
      
    # label of x-axis 
    plt.xlabel("Time") 
     
    # actual plotting 
    plt.plot(time, signal) 
      
    # shows the plot  
    # in new window 
    plt.show() 

    
visualize('./test.wav')

In [ ]:
from scipy.io.wavfile import read
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
input_data = read("./test.wav")
audio = input_data[1]
plt.plot(audio[0:1200])
plt.ylabel("Amplitude")
plt.xlabel("Time")
plt.show()

In [ ]:
plt.plot(listams)
plt.grid

# 모듈로 함수들 분리  micInput.py

In [1]:
## import, variables, 
import queue, threading
import sounddevice as sd
import soundfile as sf
import time
import numpy as np
from scipy.io.wavfile import write
import speech_recognition as sr

# Initialize the Recognizer
r = sr.Recognizer()
q = queue.Queue()

silenceStart = 0
silenceEnd = 0
silenceDur = 2
silenceThr = 10

sentenceDoneEvent = threading.Event()
## global 
recorder = False
recording = False


def trigCheckFile():
    with sf.SoundFile("./trigCheck.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=micSave):
            while recording:
                file.write(q.get())
        
def micSave(indata, frames, time, status):
    q.put(indata.copy())  

def checkingTrigger(name) :
    while True :
        global recording
        global recorder
        
        recording = True
        recorder = threading.Thread(target=trigCheckFile )
        print('trigcheck : start recording')
        recorder.start()

        time.sleep(3)

        recording = False
        recorder.join()
        print('trigcheck : stop recording')

        audioFile = sr.AudioFile('./trigCheck.wav')
        try :
            with audioFile as source :
                audio = r.record(source)

            print("recognizing...")
            recognized = r.recognize_google(audio, language='ko-KR')
            print(recognized)

            if name in recognized :
                print("trigcheck : 이름 트리거")
                return True
        except :
            print("trigcheck : no input")
        return False



def inputUserSentence():
    global recording
    global silenceStart
    global silenceEnd
    isTalking = False
    
    
    with sf.SoundFile("./sentence.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=micSave):
            print("getting Sentence")

            while recording :
                audio_data = q.get()
                file.write(audio_data)

                ## 소리 볼륨 한계치 20 아래면 silence
                ams = np.sqrt(np.mean(abs(audio_data)))
                # 현재 시간
                silenceEnd = time.time()
                
                ## 문장 시작
                if not isTalking and ams > silenceThr :
                    print("sentence : sentence start")
                    isTalking = True
                    silenceStart = time.time()
                    
                ## 이야기중임. 
                elif ams > silenceThr  and isTalking :
                    silenceStart = time.time()
                    print("sentence : startTime updated")
                
                ## 문장 끝나고 dur 만큼 조용함.
                ## 
                if silenceEnd - silenceStart >= silenceDur and isTalking:
                    print(f"sentence : silence for {silenceDur} sec. end Sentence recording")
                    recording = False
                    sentenceDoneEvent.set()
                    return 
                    
                
def getSentence():
    global recorder
    global recording
    userSentence = ""
    
    recording = True
    recorder = threading.Thread(target=inputUserSentence)
    print('sentence : getSentence start recording')
    recorder.start()
    
    sentenceDoneEvent.wait()
    
    ## wav 파일 마무리될때까지 잠시 대기
    time.sleep(0.05)
    audioFile = sr.AudioFile('./sentence.wav')
    try :
        with audioFile as source :
            audio = r.record(source)

        print("wav convert to String...")
        userSentence = r.recognize_google(audio, language='ko-KR')
        print(userSentence)
        
    except Exception as e:
        print("sentenceError :", e)
        
    sentenceDoneEvent.clear()
    
    return userSentence
    





# 통합셀

In [ ]:
#  import micInput as mi

trigger_word = "짱구"
nameTriggered = False
userSentence = ""
getInput = False
answer = False

# 메인 프로그램
while True :

    if not nameTriggered : 
        nameTriggered = checkingTrigger(trigger_word)
        getInput = True
        
    
    elif getInput :
        print("sentence")
        userSentence = getSentence()
        answer = True
        getInput = False


    ## TODO : 유저 문장 분류
    
    
    ## TODO : 유저에게 답 chatGPT
    if answer : 
        print("answer from chatGPT")
        answer = False
        getInput = True
    
    
    ## TODO : 

    
    
    
#모듈
# while True :

#     if not nameTriggered : 
#         nameTriggered = mi.checkingTrigger(trigger_word)
#         getInput = True
        
    
#     elif getInput :
#         print("sentence")
#         userSentence = mi.getSentence()
#         print(f"useroutput1 = {userSentence}")
#         getInput = False
#         print(f"useroutput2 = {userSentence}")
        
        

#     ## 유저 





trigcheck : start recording
trigcheck : stop recording
recognizing...
trigcheck : no input
trigcheck : start recording
trigcheck : stop recording
recognizing...
지금 이제 말 안 했으니까
trigcheck : start recording
trigcheck : stop recording
recognizing...
trigcheck : no input
trigcheck : start recording
trigcheck : stop recording
recognizing...
trigcheck : no input
trigcheck : start recording
trigcheck : stop recording
recognizing...
trigcheck : no input
trigcheck : start recording
trigcheck : stop recording
recognizing...
짱구
trigcheck : 이름 트리거
sentence
sentence : getSentence start recording
getting Sentence
sentence : sentence start
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime

sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : startTime updated
sentence : sta